<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/bugfix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly==5.8.0

In [13]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata
import ipywidgets as w
from IPython.display import display
import datetime
import plotly.express as px
import plotly
import plotly.graph_objects as go


engine = create_engine(userdata.get('supabase'), client_encoding='utf8', )
df = pd.DataFrame()

In [11]:
def linechart_query_plot(data_frame: pd.DataFrame, **kwargs) -> go.Figure:
    """Creates a line chart showing queries by hour for each platform."""
    fig = px.line(
        data_frame,
        x='hour',  # Hour of the day
        y='count',  # Query count
        color='query',  # Different queries
        facet_row='platform',  # Split by platform
        **kwargs
    )

    # Add plot title and adjust layout
    fig.update_layout(
        title={
            'text': "Dynamics in number of popular search queries by hour",  # Title
            'x': 0.5,  # Center title
            'xanchor': 'center',

        },
        legend=dict(
            orientation="h",  # Horizontal legend
            xanchor="center",  # Center the legend
            x=0.5
        ),

    )
    # Automatically adjust subplot titles based on facet_row
    fig.for_each_annotation(lambda a: a.update(text=f"Platform: {a.text}"))
    fig.update_layout(
        margin=dict(l=20, r=20, t=30, b=20),
    )
    for axis in fig.layout:
        if axis.startswith("yaxis"):
            fig.layout[axis].matches = None  # Remove matching for y-axes

    return fig

In [25]:
@w.interact_manual(start_date=w.DatePicker(description='Start Date', disabled=False, value = datetime.date(2021,9,1)),
                  end_date = w.DatePicker(description='End Date', disabled=False, value = datetime.date(2021,9,21)),
                  limit = w.IntText(description='Limit', value=0)
                  )
def select(start_date, end_date, limit):
    SQL_TEMPLATE_QUERY_LINECHART = """
    WITH t AS (
        SELECT
            query,
            ROW_NUMBER() OVER (PARTITION BY date_part('hour', ts) ORDER BY COUNT(*) DESC) AS rnb
        FROM
            vizro.yandex_data yd
        WHERE
            ts BETWEEN '{start_date}' AND '{end_date}'
        GROUP BY query, date_part('hour', ts)
    )
    SELECT
        platform,
        date_part('hour', ts)::INT AS hour,
        query,
        COUNT(*)
    FROM
        vizro.yandex_data yd
    WHERE
        query IN (SELECT DISTINCT query FROM t WHERE rnb < 6)
    GROUP BY
        platform, date_part('hour', ts), query
        {limit}
    """.format(start_date=start_date, end_date=end_date, limit=f'LIMIT {limit}' if limit>0 else '')
    sql = text(SQL_TEMPLATE_QUERY_LINECHART)
    global df
    df = pd.read_sql(sql, engine).sort_values(by='hour')#.reset_index(drop=True)
    linechart_query_plot(df).show()


interactive(children=(DatePicker(value=datetime.date(2021, 9, 1), description='Start Date'), DatePicker(value=…

In [24]:
df.query('query=="таблица менделеева"').sort_values(by='hour')

,platform,hour,query,count
202,desktop,0,таблица менделеева,29
587,touch,0,таблица менделеева,34
660,desktop,1,таблица менделеева,19
355,touch,1,таблица менделеева,17
200,touch,2,таблица менделеева,6
111,desktop,2,таблица менделеева,8
382,touch,3,таблица менделеева,15
290,desktop,3,таблица менделеева,9
401,touch,4,таблица менделеева,25
644,desktop,4,таблица менделеева,19


In [18]:
df

,platform,hour,query,count
0,touch,20,таблица менделеева,111
1,desktop,10,хентай,31
2,desktop,1,1 сентября,1
3,desktop,0,с днем знаний,5
4,touch,9,хентай,169
...,...,...,...,...
705,desktop,1,таблица менделеева,19
706,touch,20,1 сентября,14
707,desktop,10,секс,53
708,desktop,14,с 1 сентября,7
